# What would be my favorite city to relocate from Jersey City?
Let me postulate why I love Jersey City, and let's find out which city has close/better semblance to my city.<br>
I am a **coffee** person, and I would take a city that has lot's of coffee shops than anything else. However, lets not draw conclusions based only on coffee, as there are more important things to consider other than coffee. I would also love **grocery** shopping..... Whether I love it or not, I have to shop groceries, once every three days. I am a tea-toddler, and I would love a locality which has **less** number of **Pubs, and Bars**. I would also like to have a **pizza** once a while. I also love visiting **restaraunts** once a while. Well, not to mention, **icecreams**, one of my favorites. <br>
So, these are my want lists. Let's check out, which city in United States, satisfies my needs. 

In [2]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import folium
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

## About the Data
In order to get the list of cities in New York, New Jersey, Connecticut, Pennsylvania, we use the wiki page https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population. The wiki page, lists all the citites in Unites States. We will perform data cleaning to extract cities from aforementioned states.

### Web scraping using Beautiful Soup
Well, I am considering 4 states in United States (New York, New Jersey, Connecticut, and Pennsylvania). I need to extract all the cities from these states. We could get list of all stats form "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population". Using Beautifulsoup to scrape cities from the above mentioned wiki page.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
lis = pd.read_html(url)[4]
df = pd.DataFrame(lis)
df.head()


,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City[d],New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",".mw-parser-output .geo-default,.mw-parser-outp..."
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


### Data Cleaning
We would require only cities and states from the above dataframe. We would need latitude and longitudes of the cities as well. I feel skepticaal about the location here. Hence, lets use the geocoder library to find latitudes and longitudes.

In [4]:
df = df[['City', 'State[c]']]
df = df.rename(columns = {'State[c]': 'State'})
df.head()

,City,State
0,New York City[d],New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston[3],Texas
4,Phoenix,Arizona


Well, lets remove the extra texts which act as super-scripts in wiki page. 

In [5]:
for col in df:
    df[col] = df[col].str.split('[').str[0]
df.head(2)

,City,State
0,New York City,New York
1,Los Angeles,California


In [6]:
df = df.loc[((df['State'] == 'New York') | (df['State'] == 'New Jersey') | (df['State'] == 'Pennsylvania') | (df['State'] == 'Connecticut'))]
df.append(pd.Series({'City':'Toronto', 'State':'Ontario'}), ignore_index=True)
df

,City,State
0,New York City,New York
5,Philadelphia,Pennsylvania
65,Pittsburgh,Pennsylvania
72,Newark,New Jersey
79,Jersey City,New Jersey
85,Buffalo,New York
110,Rochester,New York
116,Yonkers,New York
180,Paterson,New Jersey
182,Bridgeport,Connecticut


###  Using Geocoder to get Latitude and Longitude
Let's get the latitudes and longitudes of every city and append it to the dataframe

In [7]:
# Getting latitude and longitude for a city
address = 'New York City, New York'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [8]:
# Method to get latitude and longitude for a series of dataframe values
def get_lat_lng(city, state):
    address = city + ', '+state
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return (latitude, longitude)

In [9]:
#Calling the get_lat_lng method to get lat and lng
loc_df = df.apply(lambda x: get_lat_lng(x.City, x.State), axis = 1)

In [10]:
#Adding the lat and lng values to the dataframe
df['Latitude'] = pd.Series(np.random.randn(), index=df.index)
df['Longitude'] = pd.Series(np.random.randn(), index=df.index)
for index, i in loc_df.items():
    df['Latitude'][index] = i[0]
    df['Longitude'][index] = i[1]

df.head(20)        

<ipython-input-10-85128e10320e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'][index] = i[0]
<ipython-input-10-85128e10320e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'][index] = i[1]


,City,State,Latitude,Longitude
0,New York City,New York,40.712728,-74.006015
5,Philadelphia,Pennsylvania,39.952724,-75.163526
65,Pittsburgh,Pennsylvania,40.441694,-79.990086
72,Newark,New Jersey,40.735657,-74.172367
79,Jersey City,New Jersey,40.728158,-74.077642
85,Buffalo,New York,42.886717,-78.878392
110,Rochester,New York,43.157285,-77.615214
116,Yonkers,New York,40.931210,-73.898747
180,Paterson,New Jersey,40.916765,-74.171811
182,Bridgeport,Connecticut,41.184722,-73.186721


### Visualizing the Cities on the map

In [11]:
latitude = 40.712728
longitude = -74.0060
cities_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cities_map)  
    
cities_map

## Using Foursquare API
We have the city names, and its latitude and longitude. Now, lets find out the trending venues at each city. We will use the Foursquare API to get the trending venues. 

Lets find the top 500 venues in each city.

In [12]:
def getVenues(city, latitude, longitude, radius = 500):
    venues_list = []
    CLIENT_ID = 'CCTF4NNBYAWDVLMNWIZ0BJCQFPH1GPULI3MAZKUKH14SMHIK'
    CLIENT_SECRET = 'JZNFEJ2Y500N12LM5H0QDZ1PSGHNMZ0P1GGLEA3XXHFV3BDD'
    VERSION = '20180605'
    LIMIT = 500
    for name, lat, lng in zip(city, latitude, longitude):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    LIMIT)
         # make the GET request
        try:
            results = requests.get(url).json()['response']['groups'][0]['items']
        except:
            continue
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return nearby_venues

In [13]:
city_venues = getVenues(city=df['City'],
                                   latitude=df['Latitude'],
                                   longitude=df['Longitude']
                                  )
city_venues

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York City,40.712728,-74.006015,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York City,40.712728,-74.006015,"The Beekman, A Thompson Hotel",40.711173,-74.006702,Hotel
2,New York City,40.712728,-74.006015,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,New York City,40.712728,-74.006015,City Hall Park,40.711893,-74.007792,Park
4,New York City,40.712728,-74.006015,Gibney Dance Center Downtown,40.713923,-74.005661,Dance Studio
...,...,...,...,...,...,...,...
629,Woodbridge,40.554180,-74.286001,Shell,40.557540,-74.284107,Gas Station
630,Woodbridge,40.554180,-74.286001,APlus at Sunoco,40.557706,-74.284776,Coffee Shop
631,Woodbridge,40.554180,-74.286001,Across the Floor,40.553987,-74.280582,Dance Studio
632,Woodbridge,40.554180,-74.286001,"Woodbridge Wine, Liquor, and Spirits",40.551399,-74.290550,Liquor Store


Lets check out how many venues does each city have

In [14]:
city_venues.groupby('City')['Venue Category'].count()

City
Allentown         29
Bridgeport         4
Buffalo           37
Elizabeth         20
Hartford          20
Jersey City       10
Lakewood          14
New Haven          4
New York City    100
Newark            64
Paterson          43
Philadelphia     100
Pittsburgh        25
Rochester         34
Stamford          30
Syracuse          33
Waterbury         22
Woodbridge        16
Yonkers           29
Name: Venue Category, dtype: int64

### Finding the most popular venues at each City
One hot encode the Venue City<br>
Find mean of each category across every City to find the top 5 venues at each city
Draw a dataframe to display the top 5 venues of every city. <br>
Note: Some venues would have None value, as its mean = 0

In [15]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['City'] = city_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,City,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,Baby Store,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,New York City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,New York City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,New York City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,New York City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,New York City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
city_grouped = city_onehot.groupby('City').mean().reset_index()
city_grouped

,City,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,Baby Store,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allentown,0.000000,0.00,0.034483,0.034483,0.00,0.034483,0.00,0.034483,0.00,...,0.000000,0.00,0.00,0.000000,0.034483,0.000000,0.000000,0.00,0.00,0.00
1,Bridgeport,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
2,Buffalo,0.027027,0.00,0.027027,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.027027,0.00,0.00,0.000000,0.000000,0.027027,0.027027,0.00,0.00,0.00
3,Elizabeth,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
4,Hartford,0.000000,0.00,0.050000,0.000000,0.00,0.000000,0.05,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
5,Jersey City,0.100000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
6,Lakewood,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
7,New Haven,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
8,New York City,0.020000,0.01,0.000000,0.000000,0.00,0.010000,0.00,0.000000,0.01,...,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.02,0.01,0.02
9,Newark,0.015625,0.00,0.031250,0.000000,0.00,0.000000,0.00,0.015625,0.00,...,0.000000,0.00,0.00,0.015625,0.000000,0.000000,0.000000,0.00,0.00,0.00


In [17]:
city_grouped = city_grouped.transpose()
new_header = city_grouped.iloc[0] #grab the first row for the header
city_grouped = city_grouped[1:] #take the data less the header row
city_grouped.columns = new_header #set the header row as the df header
city_grouped.head()

City,Allentown,Bridgeport,Buffalo,Elizabeth,Hartford,Jersey City,Lakewood,New Haven,New York City,Newark,Paterson,Philadelphia,Pittsburgh,Rochester,Stamford,Syracuse,Waterbury,Woodbridge,Yonkers
American Restaurant,0,0,0.027027,0,0,0.1,0,0,0.02,0.015625,0,0.02,0,0.0588235,0,0.0909091,0.0909091,0,0
Antique Shop,0,0,0,0,0,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0
Art Gallery,0.0344828,0.25,0.027027,0,0.05,0,0,0,0,0.03125,0,0,0,0,0,0,0,0,0
Art Museum,0.0344828,0,0,0,0,0,0,0,0,0,0,0.01,0,0,0,0.030303,0.0454545,0,0
Arts & Crafts Store,0,0,0,0,0,0,0,0,0,0,0,0.01,0,0,0,0,0,0,0


In [18]:
lis = []
lis2 = []
num_top_venues = 10
columns = []
for ind, col in city_grouped.items():
    lis.append(ind)
    sort = col.sort_values(ascending = False).head(num_top_venues)
    sort = sort[sort.values != 0]
    lis2.append(sort.index.array)
    
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

new_df2 = pd.DataFrame(lis2, columns = columns)
new_df2['City'] = lis
cols = list(new_df2.columns)
cols = [cols[-1]] + cols[:-1]
new_df2 = new_df2[cols]
new_df2.shape

(19, 11)

In [19]:
new_df2.head()

,City,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allentown,Sandwich Place,Coffee Shop,New American Restaurant,Pharmacy,Pub,Park,Café,Convenience Store,Deli / Bodega,Diner
1,Bridgeport,Park,Art Gallery,Fish Market,Donut Shop,None,None,None,None,None,None
2,Buffalo,Coffee Shop,Hotel,Sandwich Place,Bar,Pizza Place,American Restaurant,Cocktail Bar,Steakhouse,Sports Bar,Boutique
3,Elizabeth,Pizza Place,Discount Store,Coffee Shop,Theater,Burger Joint,South American Restaurant,Peruvian Restaurant,Lounge,Fried Chicken Joint,Latin American Restaurant
4,Hartford,Coffee Shop,Bar,Ice Cream Shop,Bakery,Food Court,Speakeasy,Fast Food Restaurant,New American Restaurant,Gas Station,Café


Now lets filter out the cities based on my wishlist.. 
First on the basis of coffee shops. I am eleminating any city which doesn't have coffee shops

In [20]:
new_df3 = pd.DataFrame(columns= new_df2.columns)
for i in range(new_df2.shape[0]):
    if new_df2.iloc[i,:].str.contains('Coffee').any():
        new_df3 = new_df3.append(new_df2.iloc[i,:])


In [21]:
new_df3

,City,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allentown,Sandwich Place,Coffee Shop,New American Restaurant,Pharmacy,Pub,Park,Café,Convenience Store,Deli / Bodega,Diner
2,Buffalo,Coffee Shop,Hotel,Sandwich Place,Bar,Pizza Place,American Restaurant,Cocktail Bar,Steakhouse,Sports Bar,Boutique
3,Elizabeth,Pizza Place,Discount Store,Coffee Shop,Theater,Burger Joint,South American Restaurant,Peruvian Restaurant,Lounge,Fried Chicken Joint,Latin American Restaurant
4,Hartford,Coffee Shop,Bar,Ice Cream Shop,Bakery,Food Court,Speakeasy,Fast Food Restaurant,New American Restaurant,Gas Station,Café
7,New Haven,Deli / Bodega,Convenience Store,Coffee Shop,Chinese Restaurant,None,None,None,None,None,None
8,New York City,Coffee Shop,Sandwich Place,Hotel,Gym,Burger Joint,Cocktail Bar,Italian Restaurant,Plaza,Pizza Place,Park
9,Newark,Pizza Place,Deli / Bodega,Bank,Bar,Donut Shop,Coffee Shop,Lounge,Sandwich Place,Southern / Soul Food Restaurant,Clothing Store
11,Philadelphia,Hotel,Bakery,Donut Shop,Dessert Shop,American Restaurant,Coffee Shop,Grocery Store,Gym,Spa,Deli / Bodega
12,Pittsburgh,Hotel,Coffee Shop,Lounge,Bar,Record Shop,Hockey Arena,Sandwich Place,Residential Building (Apartment / Condo),Rental Car Location,Bakery
17,Woodbridge,Dance Studio,Convenience Store,Farmers Market,Fried Chicken Joint,Liquor Store,Gas Station,Drugstore,Coffee Shop,Bakery,Donut Shop


For the reminder of  my wishlist, I am going to perform weighted voting. I cant go on with the elemination strategy, as there aren't many options left. Now, grocery stores/convinience stores, restaraunts, food truckssnadwich place, farmers market, Donut, Hotel, Bakery get additional points, and Bar/Pubs get negative points. None gets -2 points. None, just shows, how limited options, I have.

And, this is how I have made my points table. As, I love coffee the most, it goes to **Points_Table_1**. If the coffee shop exists in 1st top common venue, then it gets 12 points, if its in 2nd top common venue, it gets 10.5 points, and so on.... And, if it's not in top5 venues, it gets 1 point.

In [22]:
def points_table_1(x):
    if x == 1:
        return 12
    elif x ==2:
        return 10.5
    elif x == 3:
        return 8
    elif x == 4:
        return 5.5
    elif x == 5:
        return 3
    else:
        return 1

 As, I also like grocery, or farmers market, it goes to **Points_Table_2**. If the either of these shop exists in 1st top common venue, then it gets 10 points, if its in 2nd top common venue, it gets 10 points, and so on.... And, if it's not in top5 venues, it gets 1 point.

In [23]:
def points_table_2(x):
    if x == 1:
        return 10
    elif x ==2:
        return 8
    elif x == 3:
        return 6
    elif x == 4:
        return 4
    elif x == 5:
        return 2
    else:
        return 1

 As, I also like Restaurants, Hotels, and food, it goes to **Points_Table_3**. If the either of these shop exists in 1st top common venue, then it gets 8 points, if its in 2nd top common venue, it gets 6 points, and so on.... And, if it's not in top5 venues, it gets 0.75 point.

In [24]:
def points_table_3(x):
    if x == 1:
        return 8
    elif x ==2:
        return 6
    elif x == 3:
        return 4
    elif x == 4:
        return 2.5
    elif x == 5:
        return 1.5
    else:
        return 0.75

In [25]:
new_df3 = new_df3.reset_index()
new_df3 = new_df3.drop('index', axis = 1)

In [26]:
zero_col = np.zeros(new_df3.shape[0])
new_df3['Total'] = zero_col

In [27]:
for i in range(new_df3.shape[0]):
    for j in range(1, new_df3.shape[1]-1):       
        if new_df3.iloc[i,j] is None:
            new_df3['Total'][i] -= 2
            continue
        if 'Coffee' in new_df3.iloc[i,j]:
            new_df3['Total'][i] += points_table_1(j)
        elif (('Grocery' in new_df3.iloc[i,j]) | ('Convenience' in new_df3.iloc[i,j]) 
        | ('Market' in new_df3.iloc[i,j])):
            new_df3['Total'][i] += points_table_2(j)
        elif (('Pizza' in new_df3.iloc[i,j]) | ('Restaraunt' in new_df3.iloc[i,j]) |
        ('Hotel' in new_df3.iloc[i,j]) | ('Burger' in new_df3.iloc[i,j]) | 
        ('Food' in new_df3.iloc[i,j]) | ('Donut' in new_df3.iloc[i,j]) |
         ('Diner' in new_df3.iloc[i,j])):
            new_df3['Total'][i] += points_table_3(j)
        elif ('Bar' in new_df3.iloc[i,j]) | ('Pub' in new_df3.iloc[i,j]):
            new_df3['Total'][i] -= points_table_3(j)
            
            

<ipython-input-27-f03f7bc620af>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Total'][i] += points_table_1(j)
<ipython-input-27-f03f7bc620af>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Total'][i] -= points_table_3(j)
<ipython-input-27-f03f7bc620af>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Total'][i] += points_table_2(j)
<ipython-input-27-f03f7bc620af>:15: SettingWithCopyWarning: 
A value is

## Conclusion
Let's find out, which city suits me the best!!!

In [28]:
new_df3 = new_df3[new_df3['Total'] == new_df3['Total'].max()]

In [29]:
new_df3.to_clipboard()

### Mmm.....
We got our result. My favourite place to move out would be Buffalo or New York City. It scores the maximum points. We could see that, it has all the venues which I want, Restaurant, Coffe shop, Pizza, Hotel, and so much more, which all I need. 


### New York City or Buffalo???

Well, I would choose New York City. Why??
Because, bars are less popular in NYC than inn Buffalo. Gives me, some satisfaction, that NYC would be more peaceful. 

### Yayyy!!!!!!!! NYC